In [11]:
import pypianoroll
import os
from copy import deepcopy
import matplotlib.pyplot as plt
import numpy as np
from BinaryTokenizer import BinaryTokenizer

In [2]:
os.makedirs('data', exist_ok=True)
os.makedirs('data/pianorolls', exist_ok=True)
os.makedirs('data/midis', exist_ok=True)
os.makedirs('data/tokens', exist_ok=True)

In [3]:
midifolder = '../data/giantmidi/'
midifiles = os.listdir(midifolder)
midifile = midifiles[0]
print(midifile)

Legnani, Luigi, Rondoletto scherzoso, Op.204, iq55D8Ex4Fo.mid


In [4]:
main_piece = pypianoroll.read(midifolder + midifile)
# keep size to know when to end
main_piece_size = main_piece.downbeat.shape[0]

In [12]:
binaryTokenizer = BinaryTokenizer(num_digits=12)

In [13]:
# size in beats
segment_size = 16
start_idx = 0
end_idx = segment_size*main_piece.resolution
new_piece = deepcopy(main_piece)
new_piece.trim(start_idx,end_idx)
new_piece.binarize()
print(new_piece)
chroma = new_piece.tracks[0].pianoroll[:,:12]
for i in range(12, 128-12, 12):
    print(i)
    chroma = np.logical_or(chroma, new_piece.tracks[0].pianoroll[:,i:(i+12)])
chroma[:,-6:] = np.logical_or(chroma[:,-6:], new_piece.tracks[0].pianoroll[:,-6:])
print(chroma.shape)
indexed_chroma = binaryTokenizer.transform(chroma)
print(indexed_chroma.shape)
print(indexed_chroma)

Multitrack(name=None, resolution=24, tempo=array(shape=(384, 1), dtype=float64), downbeat=array(shape=(384, 1), dtype=bool), tracks=[BinaryTrack(name='', program=0, is_drum=False, pianoroll=array(shape=(384, 128), dtype=bool))])
12
24
36
48
60
72
84
96
108
(384, 12)
(384,)
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    8  128  128  128  128  128  128  128  128  128  128
  128  128  128  136  128  128  128  128  128  128  128  128  128  128
  128  128  128  132  132  132  132  132  128  128  128  128  128  128
  128  128  128  132  132  132  132  132  132  132  132  132  132  132
  132  132  132  133  129  129  128  128  128  128  128  128  128  128
  128  128  132    4    4    4    4    4    4    4    4    4    4    4
    4    4  133  129  129  129  129  128  128  128  128  128  128  128
  128    0    4 

In [14]:
# size in beats
segment_size = 32
piece_idx = 0
# roll
for r in range(-6,6,1):
    print(f'running for roll: {r}')
    start_idx = 0
    end_idx = segment_size*main_piece.resolution
    tmp_pianoroll = np.roll(main_piece.tracks[0].pianoroll, [0,r])
    while end_idx < main_piece_size:
        print(f'running for start: {start_idx} and end: {end_idx}')
        # create deepcopy of main piece
        new_piece = deepcopy(main_piece)
        new_piece.tracks[0].pianoroll = tmp_pianoroll
        # trim and binarize
        new_piece.trim(start_idx,end_idx)
        new_piece.binarize()
        # make chroma
        chroma = tmp_pianoroll[:,:12]
        for i in range(12, 128-12, 12):
            chroma = np.logical_or(chroma, tmp_pianoroll[:,i:(i+12)])
        chroma[:,-6:] = np.logical_or(chroma[:,-6:], tmp_pianoroll[:,-6:])
        indexed_chroma = binaryTokenizer.transform( chroma )
        # save npz
        np.savez(f'data/pianorolls/{piece_idx:05}.npz', indexed_chroma=indexed_chroma)
        # save midi
        new_piece.write(f'data/midis/{piece_idx:05}.mid')
        start_idx = end_idx
        end_idx += segment_size*main_piece.resolution
        piece_idx += 1

running for roll: -6
running for start: 0 and end: 768
running for start: 768 and end: 1536
running for start: 1536 and end: 2304
running for start: 2304 and end: 3072
running for start: 3072 and end: 3840
running for start: 3840 and end: 4608
running for start: 4608 and end: 5376
running for start: 5376 and end: 6144
running for start: 6144 and end: 6912
running for start: 6912 and end: 7680
running for start: 7680 and end: 8448
running for start: 8448 and end: 9216
running for start: 9216 and end: 9984
running for start: 9984 and end: 10752
running for start: 10752 and end: 11520
running for start: 11520 and end: 12288
running for start: 12288 and end: 13056
running for start: 13056 and end: 13824
running for start: 13824 and end: 14592
running for start: 14592 and end: 15360
running for start: 15360 and end: 16128
running for roll: -5
running for start: 0 and end: 768
running for start: 768 and end: 1536
running for start: 1536 and end: 2304
running for start: 2304 and end: 3072
run